# RAG LLM: Parsing - Chunking - Embedding - Indexing files

In this notebook it is shown how to upload a PDF document into a vector DB, mainly consists in 4 steps:

- **Parsing** is the process of extracting raw text from documents such as PDFs, .docx files, youtube videos and so on. It depends on the type of data you want to parse. For this LLM, only pdfs will be parsed.

- **Chunking** is the process of splitting the parsed text into small chunks, which will be then embedded.

- **Embedding** is the process of convertir the chunked text into a dense vector.

- **Indexing** is the process of inserting the embeddings into a vector database collection.

## Load libraries

In [1]:
import pymupdf
from langchain_text_splitters import TokenTextSplitter 
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
from qdrant_client.models import PointStruct
import numpy as np
import requests
import uuid
from datetime import datetime

import sys

sys.path.append("..")

from rag_llm_energy_expert.utils.gcp.gcs import get_file
from rag_llm_energy_expert.utils.vector_db.qdrant import create_collection, update_points
from rag_llm_energy_expert.services.ingestion.parsers.pdf_parser import parse_pdf_file
from rag_llm_energy_expert.config import QdrantConfig, GCPConfig
from rag_llm_energy_expert.credentials import generate_id_token
from rag_llm_energy_expert.services.ingestion.ingestion_pipeline import main

c:\Users\Emmanuel\Desktop\git_repos\RAG-LLM-Energy-Expert\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Initialize config classes

In [2]:
gcp_config = GCPConfig()
qdrant_config = QdrantConfig()

## Process to ingest a PDF document from zero

### Parsing PDFs

There are tons of libraries to extract data from PDFs, nevertheless, [*PyMuPDF*](https://pypi.org/project/pymupdf4llm/) is one of the best libraries because:

- Detects standard text and tables
- Header lines are identified via de font size and appropiately prefixed with one or more '#' tags.
- Bold, italic, mono-spaced text and code blocks are detected and formatted accordingly.
- By default, all document pages are processed.
- Support for pages with multiple text columns.
- Support image or vector graphic on the page and they're stored as an image.
- ***Support for page chunks***. Instead of returning one large string for the whole document, a list of dictionaries can be generated. One for each page.

*All the data parsed here comes from GCP*

Reading into memory is faster than download the pdf into a file system, and then read the file from there. Moreover, it's useful when you do not have a persistent memory or you want to work directly with the file.

### Extracting data from PDF

First, set the path to the PDF stored in GCS

In [3]:
file_to_read = f"gs://{gcp_config.BUCKET_NAME}/documents/summaries/resumen_reforma_energetica.pdf"

#local path
#file_to_read = "../data/Mexico_energy_profile.pdf"

Obtaining some metadata from the text, such as the title and the storage path that will be used as metadata within a chunk. Nevertheless, any data can be set in the metadata.

In [4]:
# Get the name / title of the file
title = file_to_read.split("/")[-1].split(".")[0]

useful_gcs_path = file_to_read[5:]

pdf_path_parts = useful_gcs_path.split("/", maxsplit= 1)
        

bucket_name = pdf_path_parts[0]
blob_name = pdf_path_parts[1]

Download in memory the PDF as bytes

In [5]:
#Loads in memory a pdf stored in GCS
pdf_bytes = get_file(gcs_file_path=blob_name, bucket_name = bucket_name)

Read the PDF using pymupdf, which creates a Document, each Document is made of Pages.

Then, using pymupdf4llm, convert the PDF text into a markdown format, returning a string with the 'markdowned' text

In [6]:
# Create a Document object, it can be constructed from a file or from memory
# pymupdf.Document() method is exactly the same as pymupdf.open()
doc = pymupdf.Document(stream = pdf_bytes)

# When comes from local:
#doc = pymupdf.Document(file_to_read)

In [7]:
pdf_text = '\n'.join([doc.get_page_text(pno = page_num) for page_num in range(len(doc))])

pdf_text

' Resumen Ejecutivo\n\n\n3\nI. Introducción\nLa Reforma Energética es un paso decidido rumbo a la modernización del sector energético de \nnuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamien-\nto de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a \nnivel legistlación secundarias, surge del estudio y valoración de las distintas iniciativas presenta-\ndas por los partidos políticos representados en el Congreso.\nLa Reforma Energética tiene los siguientes objetivos y premisas fundamentales:\n1.\t\nMantener la propiedad de la Nación sobre los hidrocarburos que se encuentran en el sub-\nsuelo.\n2.\t\nModernizar y fortalecer, sin privatizar, a Petróleos Mexicanos (Pemex) y a la Comisión Fe-\nderal de Electricidad (CFE) como Empresas Productivas del Estado, 100% públicas y 100% \nmexicanas.\n3.\t\nReducir la exposición del país a los riesgos financieros, geológicos y ambientales en las ac-\ntividades de exp

### Chunking

**Chunking depends of how many tokens an embedding model supports**

There's a [HuggingFace dashboard](https://huggingface.co/spaces/mteb/leaderboard) that compares the performance of different embedding models. Some metrics to focus on are:

- Number of Parameters: 

    A higher value means the model requires more CPU/GPU memory to run

- Embedding Dimension:

    The dimension of the vectors produced

- Max tokens:

    How many tokens the model can process, the higher the better.


For this time, we'll be using the [*sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2*](https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2) model:

- Number of Parameters: 118M
- Embedding Dimension: 768
- Max tokens: 512



[*Langchain*](https://python.langchain.com/api_reference/text_splitters/base/langchain_text_splitters.base.TextSplitter.html#textsplitter) contains many *text splitters*. Most of them uses string characters to split the data, nevertheless, an *embedding model* uses *tokens* to delimit the amount of text to be embedded into a vector. To fix this, we can use a Langchain [*TokenTextSplitter*](https://python.langchain.com/api_reference/text_splitters/base/langchain_text_splitters.base.TokenTextSplitter.html) class.


The *TokenTextSplitter* object splits the text into tokens using a model tokenizer. We can use a [*HuggingFace Tokenizer*](https://huggingface.co/docs/transformers/en/main_classes/tokenizer) as an imput parameter for the TokenTextSplitter. A *tokenizer* is in charge of preparing the inputs for a model. The library contains tokenizers for all the embedding models.

In [8]:
embedding_model = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-V2"

In [9]:
#Initialize the model using the transformer library
model = SentenceTransformer(embedding_model, trust_remote_code=True)

In [10]:
# Initialize an AutoTokenizer instance, this will tokenize the data based on 
# the embedding model used
tokenizer = AutoTokenizer.from_pretrained(embedding_model)

# Create a TokenTextSplitter instance, this will use the Tokenizer to
# tokenize the text, split it based on the max tokens supported by the
# embedding model, and then convert the tokens into text again, but now splitted.
splitter = TokenTextSplitter.from_huggingface_tokenizer(
    tokenizer,
    chunk_size=model.max_seq_length,
    chunk_overlap = 0,
)

In [11]:
chunks = splitter.split_text(pdf_text)

In [12]:
chunks[:3]

[' Resumen Ejecutivo\n\n\n3\nI. Introducción\nLa Reforma Energética es un paso decidido rumbo a la modernización del sector energético de \nnuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamien-\nto de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a \nnivel legistlación secundarias, surge',
 ' del estudio y valoración de las distintas iniciativas presenta-\ndas por los partidos políticos representados en el Congreso.\nLa Reforma Energética tiene los siguientes objetivos y premisas fundamentales:\n1.\t\nMantener la propiedad de la Nación sobre los hidrocarburos que se encuentran en el sub-\nsuelo.\n2.\t\nModernizar y fortalecer, sin privatizar, a Petró',
 'leos Mexicanos (Pemex) y a la Comisión Fe-\nderal de Electricidad (CFE) como Empresas Productivas del Estado, 100% públicas y 100% \nmexicanas.\n3.\t\nReducir la exposición del país a los riesgos financieros, geológicos y ambientales en las ac-\ntivid

In [13]:
chunks_sizes = [len(chunk) for chunk in chunks]

print(f"Number of chunks: {len(chunks)} \n"
      f"mean chunk size (in characters): {np.mean(chunks_sizes):.4f}\n"
      f"max chunk size (in characters): {max(chunks_sizes)}"
      )

Number of chunks: 148 
mean chunk size (in characters): 362.0068
max chunk size (in characters): 405


### Embedding

Now that the data is splitted based on the chunk size of each embedding model. We must notice that a vector DB is the place where we will store the chunks to create the semantic search, to do so, each chunk must be embedded into a vector using the embedding model that fits the chunking size.

There are 3 key elements that define a vector in a vector DB:

- ID
- Dimensions
- Payload

The payload contains all the metadata and the text that was embedded in the vector DB. This is because once the text has been encoded, you cannot retrieve the original data from the vector. So it is necessary to store the text as metadata.

In the next sections, we'll be adding metadata to the chunks created.

In [14]:
# Adding info such as title, and the date of chunking
extra_metadata = {
    "upload_date": datetime.now().strftime(r"%Y-%m-%d"),
    "title": title,
    "storage_path": file_to_read,
    }

Adding the extra metadata to each chunk

In [15]:
# Updating the metadata with the extra metadata. This list will return None, because the update
# method does not return the dictionary itself
final_chunks = [{"text": chunk_text, "metadata": extra_metadata,} for chunk_text in chunks]

final_chunks[:2]

[{'text': ' Resumen Ejecutivo\n\n\n3\nI. Introducción\nLa Reforma Energética es un paso decidido rumbo a la modernización del sector energético de \nnuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamien-\nto de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a \nnivel legistlación secundarias, surge',
  'metadata': {'upload_date': '2025-04-16',
   'title': 'resumen_reforma_energetica',
   'storage_path': 'gs://rag_llm_energy_expert/documents/summaries/resumen_reforma_energetica.pdf'}},
 {'text': ' del estudio y valoración de las distintas iniciativas presenta-\ndas por los partidos políticos representados en el Congreso.\nLa Reforma Energética tiene los siguientes objetivos y premisas fundamentales:\n1.\t\nMantener la propiedad de la Nación sobre los hidrocarburos que se encuentran en el sub-\nsuelo.\n2.\t\nModernizar y fortalecer, sin privatizar, a Petró',
  'metadata': {'upload_date': '2025-04-16',
   

Now we will use batch embedding to create the vectors for each chunk due to its faster than creating them one by one

In [16]:
# Creating a list of chunk's data to be embedded into vectors
chunk_text = [chunk_data["text"] for chunk_data in final_chunks]

vectors = model.encode(chunk_text)

Now we weill create a PointStruct object for each chunk

Each PointStruct has:

- id: uuid string or integer
- vector: Embedding data
- payload: all the metadata associated to the vector

In [17]:
final_chunks[0]

{'text': ' Resumen Ejecutivo\n\n\n3\nI. Introducción\nLa Reforma Energética es un paso decidido rumbo a la modernización del sector energético de \nnuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamien-\nto de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a \nnivel legistlación secundarias, surge',
 'metadata': {'upload_date': '2025-04-16',
  'title': 'resumen_reforma_energetica',
  'storage_path': 'gs://rag_llm_energy_expert/documents/summaries/resumen_reforma_energetica.pdf'}}

In [18]:
points = [PointStruct(
            id = str(uuid.uuid4()),
            vector = vectors[x],
            payload = final_chunks[x],
        )
        for x in range(len(final_chunks))]

In [19]:
points[0].payload

{'text': ' Resumen Ejecutivo\n\n\n3\nI. Introducción\nLa Reforma Energética es un paso decidido rumbo a la modernización del sector energético de \nnuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamien-\nto de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a \nnivel legistlación secundarias, surge',
 'metadata': {'upload_date': '2025-04-16',
  'title': 'resumen_reforma_energetica',
  'storage_path': 'gs://rag_llm_energy_expert/documents/summaries/resumen_reforma_energetica.pdf'}}

### Indexing

In this last step, all the embeddings generated are stored into a vector DB collection

In [20]:
# Creating a collection inside the vector db
collection_name=qdrant_config.COLLECTION_NAME + qdrant_config.COLLECTION_VERSION

In [21]:
# This step is optional
create_collection(
    collection_name=collection_name, 
    vector_size=model[1].word_embedding_dimension, # Length of the vector
    )

# Update points is capable of update documents and upload a new one
update_points(
    collection_name=collection_name,
    points=points,
)

2025-04-16 11:39:42.863 | INFO     | rag_llm_energy_expert.utils.vector_db.qdrant:create_collection:200 - Creating collection...


2025-04-16 11:39:43.117 | INFO     | rag_llm_energy_expert.utils.vector_db.qdrant:create_collection:212 - The collection already exists
2025-04-16 11:39:43.132 | INFO     | rag_llm_energy_expert.utils.vector_db.qdrant:update_points:160 - Updating points...
2025-04-16 11:39:43.305 | INFO     | rag_llm_energy_expert.utils.vector_db.qdrant:delete_document:85 - Deleting previous points in the Vector DB related to the document...
2025-04-16 11:39:43.537 | INFO     | rag_llm_energy_expert.utils.vector_db.qdrant:delete_document:105 - Document resumen_reforma_energetica deleted
2025-04-16 11:39:43.537 | INFO     | rag_llm_energy_expert.utils.vector_db.qdrant:upload_points:119 - Uploading points...
2025-04-16 11:39:44.603 | INFO     | rag_llm_energy_expert.utils.vector_db.qdrant:upload_points:146 - Points uploaded into the collection energy_expert_v1


## Loading a PDF using the Embedding Service

### Extract PDF text

In [22]:
file_to_read = f"gs://{gcp_config.BUCKET_NAME}/documents/summaries/resumen_reforma_energetica.pdf"
pdf_info = parse_pdf_file(pdf_path = file_to_read)

2025-04-16 11:39:44.634 | INFO     | rag_llm_energy_expert.services.ingestion.parsers.pdf_parser:parse_pdf_file:28 - Extracting PDF content...
2025-04-16 11:39:44.640 | INFO     | rag_llm_energy_expert.services.ingestion.parsers.pdf_parser:parse_pdf_file:44 - Google cloud storage path detected
2025-04-16 11:39:45.062 | INFO     | rag_llm_energy_expert.services.ingestion.parsers.pdf_parser:parse_pdf_file:94 - PDF content successfully extracted


In [23]:
pdf_info

{'text': ' Resumen Ejecutivo\n\n\n3\nI. Introducción\nLa Reforma Energética es un paso decidido rumbo a la modernización del sector energético de \nnuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamien-\nto de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a \nnivel legistlación secundarias, surge del estudio y valoración de las distintas iniciativas presenta-\ndas por los partidos políticos representados en el Congreso.\nLa Reforma Energética tiene los siguientes objetivos y premisas fundamentales:\n1.\t\nMantener la propiedad de la Nación sobre los hidrocarburos que se encuentran en el sub-\nsuelo.\n2.\t\nModernizar y fortalecer, sin privatizar, a Petróleos Mexicanos (Pemex) y a la Comisión Fe-\nderal de Electricidad (CFE) como Empresas Productivas del Estado, 100% públicas y 100% \nmexicanas.\n3.\t\nReducir la exposición del país a los riesgos financieros, geológicos y ambientales en las ac-\ntividad

### Generate Embeddings

In [24]:
token = generate_id_token(audience=gcp_config.EMBEDDING_SERVICE_URL)

headers = {"Authorization": f"Bearer {token}"}

response = requests.post(url=gcp_config.EMBEDDING_SERVICE_URL + "/embed-text", json=pdf_info, headers=headers)

In [25]:
embeddings = response.json()["chunks"]
vector_dimension = len(embeddings[0]['vector'])

In [26]:
print(f"Chunks generated: {len(embeddings)}")
print(f"Embedding dimension: {vector_dimension}")

Chunks generated: 148
Embedding dimension: 384


In [27]:
embeddings

[{'vector_id': '3c02eae5-b9e3-440e-ad41-2b1d2e3ac73a',
  'vector': [-0.11057437211275101,
   0.21187715232372284,
   0.014034171588718891,
   0.1121615394949913,
   0.18197445571422577,
   0.06306228041648865,
   -0.22808468341827393,
   -0.0837114006280899,
   -0.03658006712794304,
   0.1671772301197052,
   0.07837821543216705,
   0.16965468227863312,
   0.0358286015689373,
   -0.18512898683547974,
   0.001741167507134378,
   0.024766137823462486,
   -0.20692230761051178,
   -0.1459364891052246,
   -0.03757733479142189,
   0.010992496274411678,
   0.15808364748954773,
   -0.09585031867027283,
   0.03204415738582611,
   -0.07978127896785736,
   0.12208936363458633,
   -0.04470371454954147,
   0.05382337421178818,
   -0.016607481986284256,
   -0.06273837387561798,
   -0.24737101793289185,
   0.0035479136276990175,
   0.04342753067612648,
   -0.009677430614829063,
   0.11098742485046387,
   0.030203377828001976,
   -0.09622641652822495,
   0.08192652463912964,
   0.037369612604379654,
  

### Insert chunks into a vector DB

In [28]:
points = [PointStruct(
            id = embeddings[x]["vector_id"],
            vector = embeddings[x]["vector"],
            payload = embeddings[x]["payload"],
        )
        for x in range(len(embeddings))]

In [29]:
points[0]

PointStruct(id='3c02eae5-b9e3-440e-ad41-2b1d2e3ac73a', vector=[-0.11057437211275101, 0.21187715232372284, 0.014034171588718891, 0.1121615394949913, 0.18197445571422577, 0.06306228041648865, -0.22808468341827393, -0.0837114006280899, -0.03658006712794304, 0.1671772301197052, 0.07837821543216705, 0.16965468227863312, 0.0358286015689373, -0.18512898683547974, 0.001741167507134378, 0.024766137823462486, -0.20692230761051178, -0.1459364891052246, -0.03757733479142189, 0.010992496274411678, 0.15808364748954773, -0.09585031867027283, 0.03204415738582611, -0.07978127896785736, 0.12208936363458633, -0.04470371454954147, 0.05382337421178818, -0.016607481986284256, -0.06273837387561798, -0.24737101793289185, 0.0035479136276990175, 0.04342753067612648, -0.009677430614829063, 0.11098742485046387, 0.030203377828001976, -0.09622641652822495, 0.08192652463912964, 0.037369612604379654, 0.08715157210826874, 0.1488027125597, -0.05854753404855728, -0.3493401110172272, 0.03462224826216698, -0.0992706865072

In [30]:
# Creating a collection inside the vector db
collection_name=qdrant_config.COLLECTION_NAME + qdrant_config.COLLECTION_VERSION

In [31]:
# This step is optional
create_collection(
    collection_name=collection_name, 
    vector_size=vector_dimension, # Length of the vector
    )

# Update points is capable of update documents and upload a new one
update_points(
    collection_name=collection_name,
    points=points,
)

2025-04-16 11:40:08.219 | INFO     | rag_llm_energy_expert.utils.vector_db.qdrant:create_collection:200 - Creating collection...
2025-04-16 11:40:08.467 | INFO     | rag_llm_energy_expert.utils.vector_db.qdrant:create_collection:212 - The collection already exists
2025-04-16 11:40:08.467 | INFO     | rag_llm_energy_expert.utils.vector_db.qdrant:update_points:160 - Updating points...
2025-04-16 11:40:08.636 | INFO     | rag_llm_energy_expert.utils.vector_db.qdrant:delete_document:85 - Deleting previous points in the Vector DB related to the document...
2025-04-16 11:40:08.889 | INFO     | rag_llm_energy_expert.utils.vector_db.qdrant:delete_document:105 - Document resumen_reforma_energetica deleted
2025-04-16 11:40:08.889 | INFO     | rag_llm_energy_expert.utils.vector_db.qdrant:upload_points:119 - Uploading points...
2025-04-16 11:40:09.756 | INFO     | rag_llm_energy_expert.utils.vector_db.qdrant:upload_points:146 - Points uploaded into the collection energy_expert_v1


## Loading a PDF using the Ingestion Pipeline

In [32]:
main(
    file_path=file_to_read,
    embedding_model_name=embedding_model,
    chunk_overlap=0,
    collection_name=collection_name
)

2025-04-16 11:40:09.777 | INFO     | rag_llm_energy_expert.services.ingestion.ingestion_pipeline:main:35 - Parsing file...
2025-04-16 11:40:09.781 | INFO     | rag_llm_energy_expert.services.ingestion.parsers.pdf_parser:parse_pdf_file:28 - Extracting PDF content...
2025-04-16 11:40:09.786 | INFO     | rag_llm_energy_expert.services.ingestion.parsers.pdf_parser:parse_pdf_file:44 - Google cloud storage path detected
2025-04-16 11:40:10.205 | INFO     | rag_llm_energy_expert.services.ingestion.parsers.pdf_parser:parse_pdf_file:94 - PDF content successfully extracted
2025-04-16 11:40:10.207 | INFO     | rag_llm_energy_expert.services.ingestion.ingestion_pipeline:main:52 - Generating embeddings...
2025-04-16 11:40:32.963 | INFO     | rag_llm_energy_expert.services.ingestion.ingestion_pipeline:main:82 - Embeddings generated
2025-04-16 11:40:33.045 | INFO     | rag_llm_energy_expert.utils.vector_db.qdrant:create_points:239 - Creating points...
2025-04-16 11:40:33.045 | INFO     | rag_llm_ener